In [1]:
!source activate eman-env


INFO: eman-deps activation script is making the following environmental changes:
++++ LDFLAGS=
++++ LDFLAGS=
++++ CXXFLAGS=
++++ set +x



In [2]:
!em2dam -h

Usage: em2dam [OPTIONS] <MAP> [DAT]

EM2DAM - Electron Microscopy density map to Dummy Atom Model

Known Arguments:
  MAP                        EM MAP file
  DAT                        Experimental SAXS data (optional)

Known Options:
  -t, --threshold=<X>        cutoff threshold (default: minimum density)
  -r, --reduction=<N>        reduction level, merges N^3 beads
      --maxgraphs=<N>        maximum number of graphs in output (default: 1)
      --maxbeads=<N>         maximum number of beads in output (default: 100.000)
      --damstart             make output model suitable as search volume for dammin
      --lm=<N>               number of spherical harmonics to use (default: 15)
      --ns=<N>               number of points in scattering vector (default: 256)
      --sm=<X>               maximum scattering vector (default: 0.5 A^-1)
  -p, --prefix=<PREFIX>      output filename prefix (default: basename of input map)
  -h, --help                 Print usage information and exit
 